In [1]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import MAPS_util

## set the working directory

In [ ]:
# check current directory
os.getcwd()

In [ ]:
# change directory
os.chdir('your-directory')
# confirm the directory
os.getcwd()

## get to work

In [ ]:
folder_path = "your-image-folder"
create_folder(folder_path, "not-blurry")

In [54]:
dst_folder = folder_path + "not-blurry"

In [ ]:
# get the file names of the images in the folder
ID_list = get_names(folder_path)

for x in ID_list:
    print('ID = {}'.format(x))
    for entry in os.scandir(folder_path): # import each channel
        if x in entry.name and entry.name.endswith('d0.TIF'):
            blue = cv2.imread(entry.path, 0)
            print(entry.name)
        if x in entry.name and entry.name.endswith('d1.TIF'):
            green = cv2.imread(entry.path, 0)
            print(entry.name)
        if x in entry.name and entry.name.endswith('d2.TIF'):
            red = cv2.imread(entry.path, 0)
            print(entry.name)
        else:
            continue
        
        # use DAPI to do 1st round of filtering out blurry images
        edgesBlue = cv2.Canny(blue, 100, 200)
        fmBlue = variance_of_laplacian(edgesBlue)
        
        if fmBlue > 800: # fine-tune this number if the default doesn't work
            print('fm = {:2f}'.format(fmBlue))
            # blur out over-exposed cells
            mask = find_blobs(red)
            blrred = blur_blobs(red, mask)
                
            # remove air bubbles
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))  # taking a matrix of size 5 as kernel
            morph = cv2.morphologyEx(blrred, cv2.MORPH_OPEN, kernel) # (erosion followed by dilation)
            
            # 2nd rnd of edge detect, after processing tricky images
            edgesRed = cv2.Canny(morph, 50, 200)
            fmRed = variance_of_laplacian(edgesRed)
            
            if fmRed > 230:
                # illumination correction on all channels
                #blue = illum_corr(blue)
                green = hist_adj(green, clahe=False, offset=0)
                red = hist_adj(red, clahe=False, offset=0)
                
                mergedImg = cv2.merge([blue, green, red])
                fName = 'merged_' + str(x) + '.TIF'
                print('Saving file - {}...'.format(fName))
                cv2.imwrite(dst_folder + fName, mergedImg)